In [4]:
import sys
!{sys.executable} -m pip install autogluon.timeseries

  Using cached gluonts-0.15.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached boto3-1.35.56-py3-none-any.whl.metadata (6.7 kB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached window_ops-0.0.15-py3-none-any.whl.metadata (6.8 kB)
  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached botocore-1.35.56-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached py4j-0.10.9

In [5]:
import torch
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

for i in range(num_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    print(f"GPU {i}: {gpu_name}")

Number of GPUs available: 1
GPU 0: NVIDIA A30


In [6]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import pandas as pd
import numpy as np

/home/yl2672496l/Yue/anaconda3/envs/llmtime/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read data

In [10]:
my_data = pd.read_csv('../GD030A_S.csv')

In [11]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [12]:
traffic_full = recover_timestamp(my_data)
traffic_full

,date,time,flow
2019-10-01 00:00:00,2019-10-01,0.0,15.0
2019-10-01 01:00:00,2019-10-01,1.0,9.0
2019-10-01 02:00:00,2019-10-01,2.0,9.0
2019-10-01 03:00:00,2019-10-01,3.0,7.0
2019-10-01 04:00:00,2019-10-01,4.0,9.0
...,...,...,...
2023-09-30 19:00:00,2023-09-30,19.0,129.0
2023-09-30 20:00:00,2023-09-30,20.0,119.0
2023-09-30 21:00:00,2023-09-30,21.0,106.0
2023-09-30 22:00:00,2023-09-30,22.0,88.0


In [13]:
train_set = traffic_full[:'2022-02-28 23:00:00']
valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
test_set = traffic_full['2023-01-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full)))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full)))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full)))

Proportion of train_set : 0.6037
Proportion of valid_set : 0.2094
Proportion of test_set : 0.1869


In [14]:
def create_sequences(data, input_length, forecast_horizon):
    """
    Creates sequences for time series data, excluding any sequences containing NaN values.

    Parameters:
    - data: pandas DataFrame containing the data. Must include the 'flow' column.
    - input_length: int, number of past time steps to include in each input sequence.
    - forecast_horizon: int, number of future steps to predict.

    Returns:
    - sequences_df: pandas DataFrame containing all sequences with input lags and output leads,
                    and an 'item_id' column to separate each sequence, including timestamps.
    """
    sequences = []
    item_id = 0  # To assign unique ID to each sequence

    for i in range(input_length, len(data) - forecast_horizon + 1):
        # Extract the input sequence
        X_seq = data.iloc[i - input_length:i]['flow'].values
        X_timestamps = data.iloc[i - input_length:i].index
        # Extract the target sequence
        y_seq = data.iloc[i:i + forecast_horizon]['flow'].values
        y_timestamps = data.iloc[i:i + forecast_horizon].index

        # Check for NaN values in the input sequence and target sequence
        if not np.isnan(X_seq).any() and not np.isnan(y_seq).any():
            # Convert X_seq and y_seq to DataFrames, including their timestamps
            x_df = pd.DataFrame({'target': X_seq, 'timestamp': X_timestamps})
            y_df = pd.DataFrame({'target': y_seq, 'timestamp': y_timestamps})

            # Concatenate X_df and y_df
            seq_df = pd.concat([x_df, y_df], ignore_index=True)

            # Add 'item_id' column
            seq_df['item_id'] = str(item_id)
            item_id += 1  # Increment item_id

            # Append to list
            sequences.append(seq_df)
        else:
            # Optionally, log or count the skipped sequences
            pass  # Simply skip sequences with NaNs

    # Concatenate all sequences into one DataFrame
    sequences_df = pd.concat(sequences)
    sequences_df['item_id'] = sequences_df['item_id'].astype('str')
    sequences_df['target'] = sequences_df['target'].astype('float32')

    return sequences_df

In [15]:
input_lengths = [24 * i for i in range(1, 22)]
prediction_length = 6

In [16]:
from collections import defaultdict
data_dict = defaultdict(dict)

for length in input_lengths:
    print(f"Processing input length: {length}")

    # Create sequences with forecast_horizon=6
    test = create_sequences(test_set, length, forecast_horizon=6)

    # Store in the dictionary
    data_dict[length]['test'] = test

    # Print shapes and ensure no NaNs
    print(f"  test shape: {test.index.max()+1}, {len(test.item_id.unique())}\n")

Processing input length: 24
  test shape: 30, 5915

Processing input length: 48
  test shape: 54, 5728

Processing input length: 72
  test shape: 78, 5560

Processing input length: 96
  test shape: 102, 5392

Processing input length: 120
  test shape: 126, 5224

Processing input length: 144
  test shape: 150, 5056

Processing input length: 168
  test shape: 174, 4888

Processing input length: 192
  test shape: 198, 4720

Processing input length: 216
  test shape: 222, 4552

Processing input length: 240
  test shape: 246, 4384

Processing input length: 264
  test shape: 270, 4216

Processing input length: 288
  test shape: 294, 4048

Processing input length: 312
  test shape: 318, 3880

Processing input length: 336
  test shape: 342, 3712

Processing input length: 360
  test shape: 366, 3544

Processing input length: 384
  test shape: 390, 3394

Processing input length: 408
  test shape: 414, 3250

Processing input length: 432
  test shape: 438, 3123

Processing input length: 456
  test

# Zero-shot

In [17]:
def chronos_zero_shot(input_lengths, prediction_length, model_type):
    results_dict = defaultdict(dict)

    for length in input_lengths:
        print(f"Processing input length: {length}")

        # prepare the data
        test_data = TimeSeriesDataFrame(data_dict[length]['test'])
        test_data_ready, test_data = test_data.train_test_split(prediction_length)

        predictor = TimeSeriesPredictor(prediction_length=prediction_length).fit(test_data_ready, presets=model_type)
        predictions = predictor.predict(test_data_ready)

        ground_truth_df = test_data.groupby(level='item_id').tail(6)
        predicted_df = predictions[['mean']]

        results_dict[length]['Predicted_Results'] = predicted_df
        results_dict[length]['Gound_Truth'] = ground_truth_df

    return results_dict

In [26]:
torch.cuda.empty_cache()

In [ ]:
# results_dict1 = chronos_zero_shot(input_lengths, prediction_length, "chronos_tiny")
results_dict2 = chronos_zero_shot(input_lengths, prediction_length, "chronos_mini")
# results_dict3 = chronos_zero_shot(input_lengths, prediction_length, "chronos_small")

Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241108_174437'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       5.64 GB / 125.71 GB (4.5%)
Disk Space Avail:   393.41 GB / 1003.85 GB (39.2%)
Setting presets to: chronos_mini

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'mini'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 141960 rows, 5915 time ser

Processing input length: 24



Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-11-08 17:44:37
Models that will be trained: ['Chronos[mini]']
Training timeseries model Chronos[mini]. 
	0.00    s     = Training runtime
Training complete. Models trained: ['Chronos[mini]']
Total runtime: 0.01 s
Best model: Chronos[mini]
Model not specified in predict, will default to the model with the best validation score: Chronos[mini]
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241108_174517'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          

Processing input length: 48



Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-11-08 17:45:17
Models that will be trained: ['Chronos[mini]']
Training timeseries model Chronos[mini]. 
	0.00    s     = Training runtime
Training complete. Models trained: ['Chronos[mini]']
Total runtime: 0.01 s
Best model: Chronos[mini]
Model not specified in predict, will default to the model with the best validation score: Chronos[mini]
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241108_174559'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          

...

Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241108_174826'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       5.48 GB / 125.71 GB (4.4%)
Disk Space Avail:   393.37 GB / 1003.85 GB (39.2%)
Setting presets to: chronos_mini

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'mini'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 728064 rows, 5056 time ser

Processing input length: 168


Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241108_174926'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       5.38 GB / 125.71 GB (4.3%)
Disk Space Avail:   393.36 GB / 1003.85 GB (39.2%)
Setting presets to: chronos_mini

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'mini'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 821184 rows, 4888 time ser

Processing input length: 192


Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241108_175029'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       5.49 GB / 125.71 GB (4.4%)
Disk Space Avail:   393.35 GB / 1003.85 GB (39.2%)
Setting presets to: chronos_mini

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'mini'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 906240 rows, 4720 time ser

In [21]:
def cal_metrics(df_true, df_pred):
    df_merged = pd.merge(df_true, df_pred, on=['item_id', 'timestamp'], how='inner')

    # Step 3: Assign prediction step numbers (1 to 6) within each 'item_id'
    df_merged['step'] = df_merged.groupby('item_id').cumcount() + 1

    # Step 4: Calculate error metrics
    df_merged['error'] = df_merged['mean'] - df_merged['target']
    df_merged['abs_error'] = df_merged['error'].abs()
    df_merged['squared_error'] = df_merged['error'] ** 2

    # Handle division by zero in MAPE calculation by adding a very small number (epsilon)
    epsilon = 1e-10
    df_merged['abs_percentage_error'] = df_merged['abs_error'] / (df_merged['target'].abs() + epsilon)

    # Step 5: Calculate overall metrics
    overall_mse = df_merged['squared_error'].mean()
    overall_rmse = np.sqrt(overall_mse)
    overall_mae = df_merged['abs_error'].mean()
    overall_mape = df_merged['abs_percentage_error'].mean() * 100

    print("Overall Evaluation Metrics:")
    print(f"MSE: {overall_mse:.4f}")
    print(f"RMSE: {overall_rmse:.4f}")
    print(f"MAE: {overall_mae:.4f}")
    print(f"MAPE: {overall_mape:.2f}%")

    # Step 6: Calculate metrics for each prediction step
    metrics_by_step = df_merged.groupby('step').agg({
      'squared_error': 'mean',
      'abs_error': 'mean',
      'abs_percentage_error': 'mean'
    }).reset_index()

    metrics_by_step['MSE'] = metrics_by_step['squared_error']
    metrics_by_step['RMSE'] = np.sqrt(metrics_by_step['squared_error'])
    metrics_by_step['MAE'] = metrics_by_step['abs_error']
    metrics_by_step['MAPE'] = metrics_by_step['abs_percentage_error'] * 100

    # Select relevant columns and round for better readability
    metrics_by_step = metrics_by_step[['step', 'MSE', 'RMSE', 'MAE', 'MAPE']]

    return metrics_by_step

In [24]:
def metrics_results(results_dict):
    for length in input_lengths:
        print(f"Processing input length: {length}")
    
        predict = results_dict[length]['Predicted_Results']
        truth = results_dict[length]['Gound_Truth']
        evaluation_df = cal_metrics(predict, truth)
        display(evaluation_df)

In [25]:
metrics_results(results_dict1)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 2244.3584
RMSE: 47.3747
MAE: 32.5315
MAPE: 34.92%


,step,MSE,RMSE,MAE,MAPE
0,1,1298.925490,36.040609,24.836219,25.205852
1,2,1872.780967,43.275639,29.960267,30.572911
2,3,2218.228326,47.098071,32.453996,33.784616
3,4,2444.599725,49.442894,34.219329,36.914544
4,5,2688.532095,51.851057,36.071117,40.515557
5,6,2943.084039,54.250199,37.648348,42.549825


Processing input length: 48
Overall Evaluation Metrics:
MSE: 1983.0282
RMSE: 44.5312
MAE: 29.4247
MAPE: 31.65%


,step,MSE,RMSE,MAE,MAPE
0,1,1228.491676,35.049846,23.609824,24.053533
1,2,1692.785605,41.143476,27.313469,28.002762
2,3,1962.360583,44.298539,29.500414,31.049345
3,4,2160.769397,46.484077,30.917706,33.515965
4,5,2348.487830,48.461199,32.146311,36.051962
5,6,2505.274328,50.052715,33.060712,37.223322


Processing input length: 72
Overall Evaluation Metrics:
MSE: 1942.6707
RMSE: 44.0757
MAE: 28.8227
MAPE: 30.52%


...

,step,MSE,RMSE,MAE,MAPE
0,1,1264.968015,35.566389,23.156213,24.350749
1,2,1652.333501,40.648905,25.504075,26.445103
2,3,1858.085688,43.105518,26.743992,27.989781
3,4,2002.117203,44.745024,27.900457,29.046560
4,5,2204.218185,46.949102,28.776862,29.816418
5,6,2385.040383,48.836875,29.482630,30.216758


Processing input length: 480
Overall Evaluation Metrics:
MSE: 1842.4510
RMSE: 42.9238
MAE: 26.7291
MAPE: 28.31%


,step,MSE,RMSE,MAE,MAPE
0,1,1257.523947,35.461584,23.087470,24.961110
1,2,1616.415918,40.204675,25.323669,26.610397
2,3,1772.095518,42.096265,26.394914,27.969531
3,4,1933.834970,43.975391,27.699889,29.379341
4,5,2137.631901,46.234531,28.541039,30.215299
5,6,2337.203824,48.344636,29.327807,30.705733


Processing input length: 504
Overall Evaluation Metrics:
MSE: 1791.0273
RMSE: 42.3205
MAE: 26.3427
MAPE: 28.33%


,step,MSE,RMSE,MAE,MAPE
0,1,1196.260967,34.587006,22.731666,24.706446
1,2,1549.590049,39.364833,24.911869,26.876933
2,3,1743.809106,41.758940,26.195813,28.279447
3,4,1905.411670,43.651021,27.384830,29.438044
4,5,2073.165488,45.532027,28.004695,30.075250
5,6,2277.926480,47.727628,28.827535,30.600878
